# DB Queries with Django ORM

In [88]:
from django.db import connection, reset_queries
from datetime import date
# from django.db.models import Avg, Count, Min, Max, Q, F, Value as V, Exists, ..
from django.db.models import Value as V
from django.db.models.functions import Upper, Lower, Substr, Coalesce, Extract, Concat, Round, Ceil, Floor #, ...

## 1 - Basic CRUD

### 1.1 - Get all elements and pagination

In [2]:
reset_queries()
queryset = Movie.objects.all()
print(connection.queries) # no SQL here, QuerySet object is lazy
queryset

[]


<QuerySet [#5257 - Excuse Me (1915), #6366 - Anton the Terrible (1916), #8351 - The Mysterious Mrs. Musslewhite (1917), #8384 - On the Level (1917), #8642 - The Sunset Trail (1917), #8676 - The Tides of Barnegat (1917), #8981 - The Cruise of the Make-Believes (1918), #9124 - Good Night, Paul (1918), #16251 - Proud Flesh (1925), #19588 - A Woman Against the World (1928), #8079250 - Oppenheimer (2023), #9274 - A Lady's Name (1918), #9403 - Mrs. Leffingwell's Boots (1918), #9475 - A Pair of Silk Stockings (1918), #9492 - A Petticoat Pilot (1918), #9576 - Sauce for the Goose (1918), #9666 - Such a Little Pirate (1918), #9740 - Unclaimed Goods (1918), #9767 - Viviette (1918), #19699 - The Black Watch (1929), '...(remaining elements truncated)...']>

In [3]:
connection.queries

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" LIMIT 21',
  'time': '0.032'}]

In [4]:
reset_queries()
queryset = Movie.objects.all()
movie_slice = queryset[50:100]  # new lazy Query Set, pagination
movie_slice

<QuerySet [#13592 - Shadows (1922), #13611 - Smilin' Through (1922), #13751 - When Love Comes (1922), #16514 - The Wheel (1925), #21015 - Juno and the Paycock (1930), #21128 - Mary (1931), #13891 - Bright Lights of Broadway (1923), #14205 - Little Old New York (1923), #14264 - Maytime (1923), #14572 - Vanity Fair (1923), #14689 - The Average Woman (1924), #14918 - A Fool's Awakening (1924), #15022 - Janice Meredith (1924), #15249 - The Price of a Party (1924), #16048 - Lovers in Quarantine (1925), #16062 - The Mad Marriage (1925), #16093 - The Marriage Whirl (1925), #16127 - The Mountain Eagle (1926), #16230 - The Pleasure Garden (1925), #8079253 - Oppenheimer (2023), '...(remaining elements truncated)...']>

In [5]:
connection.queries

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" LIMIT 21 OFFSET 50',
  'time': '0.016'}]

In [6]:
movies = list(queryset) # a queryset is iterable
print(connection.queries)
print(len(movies), movies[:2], movies[-2:])

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" LIMIT 21 OFFSET 50', 'time': '0.016'}, {'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie"', 'time': '0.125'}]
1190 [#5257 - Excuse Me (1915), #6366 - Anton the Terrible (1916)] [#7959026 - The Mule (2018), #8079248 - Yesterday (2019)]


### 1.2 - Get One element
Only one result: primary key, unique column

In [7]:
m = Movie.objects.get(id=5257)  # pk by column name
m

#5257 - Excuse Me (1915)

In [8]:
m = Movie.objects.get(pk=5257) # pk without the name of the column
m

#5257 - Excuse Me (1915)

In [9]:
try:
    Movie.objects.get(pk=0)
except Movie.DoesNotExist:
    print("No movie with this id")

No movie with this id


### 1.3 - Lifecycle

In [10]:
reset_queries()
m = Movie(title="Oppenheimer", year=2023)
print("movie id:", m.id) 
m.save()
print("movie id:", m.id)

movie id: None
movie id: 8079254


In [11]:
connection.queries

[{'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Oppenheimer\', 2023, NULL, NULL, NULL, NULL, NULL) RETURNING "movieapp_movie"."id"',
  'time': '0.032'}]

In [12]:
m.delete()

(1, {'movieapp.Movie': 1})

In [13]:
connection.queries

[{'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Oppenheimer\', 2023, NULL, NULL, NULL, NULL, NULL) RETURNING "movieapp_movie"."id"',
  'time': '0.032'},
 {'sql': 'DELETE FROM "movieapp_movie" WHERE "movieapp_movie"."id" IN (8079254)',
  'time': '0.015'}]

In [14]:
reset_queries()
movieOppenheimer = Movie.objects.create(title="Oppenheimer", year=2023, duration=180, pg='R')
print("movie id:", movieOppenheimer.id)
connection.queries

movie id: 8079255


[{'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Oppenheimer\', 2023, 180, \'R\', NULL, NULL, NULL) RETURNING "movieapp_movie"."id"',
  'time': '0.062'}]

In [15]:
reset_queries()
movieOppenheimer.synopsis = "The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb."
movieOppenheimer.save()
connection.queries

[{'sql': 'UPDATE "movieapp_movie" SET "title" = \'Oppenheimer\', "year" = 2023, "duration" = 180, "pg" = \'R\', "synopsis" = \'The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb.\', "poster_uri" = NULL, "director_id" = NULL WHERE "movieapp_movie"."id" = 8079255',
  'time': '0.032'}]

# 2 - ManyToOne CRUD

In [16]:
reset_queries()
m = Movie.objects.get(pk=7959026)
print(connection.queries)
m

[{'sql': 'SELECT "movieapp_movie"."id", "movieapp_movie"."title", "movieapp_movie"."year", "movieapp_movie"."duration", "movieapp_movie"."pg", "movieapp_movie"."synopsis", "movieapp_movie"."poster_uri", "movieapp_movie"."director_id" FROM "movieapp_movie" WHERE "movieapp_movie"."id" = 7959026 LIMIT 21', 'time': '0.016'}]


#7959026 - The Mule (2018)

In [17]:
m.director_id

142

In [18]:
reset_queries()
d = m.director # lazy attribute is filled here (query SQL)
print(connection.queries)
d

[{'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."id" = 142 LIMIT 21', 'time': '0.000'}]


#142 - Clint Eastwood

In [19]:
print(d)

#142 - Clint Eastwood


In [20]:
nolansQuerySet = Person.objects.filter(name = "Christopher Nolan")
nolans = list(nolansQuerySet)
nolans

[#634240 - Christopher Nolan]

In [22]:
reset_queries()
nolan = nolans[0]
movieOppenheimer.director = nolan # set object relationship
movieOppenheimer.save()
connection.queries

[{'sql': 'UPDATE "movieapp_movie" SET "title" = \'Oppenheimer\', "year" = 2023, "duration" = 180, "pg" = \'R\', "synopsis" = \'The story of American scientist, J. Robert Oppenheimer, and his role in the development of the atomic bomb.\', "poster_uri" = NULL, "director_id" = 634240 WHERE "movieapp_movie"."id" = 8079255',
  'time': '0.031'}]

In [24]:
# create movie BArbie, 2023 by Greta Gerwig Born August 4, 1983
reset_queries()
greta = Person.objects.create(name="Greta Gerwig", birthdate=date(1983,8,4))
movieBarbie = Movie.objects.create(title="Barbie", year=2023, director=greta)
connection.queries

[{'sql': 'INSERT INTO "person" ("name", "birthdate") VALUES (\'Greta Gerwig\', \'1983-08-04\'::date) RETURNING "person"."id"',
  'time': '0.031'},
 {'sql': 'INSERT INTO "movieapp_movie" ("title", "year", "duration", "pg", "synopsis", "poster_uri", "director_id") VALUES (\'Barbie\', 2023, NULL, NULL, NULL, NULL, 11903874) RETURNING "movieapp_movie"."id"',
  'time': '0.047'}]

## 3 - Queries

### 3.1. Filters (Where)

In [26]:
# exact predicate (=)
Movie.objects.filter(title='The Man Who Knew Too Much')

<QuerySet [#25452 - The Man Who Knew Too Much (1934), #49470 - The Man Who Knew Too Much (1956)]>

In [27]:
# 2 predicates AND
Movie.objects.filter(title='The Man Who Knew Too Much', year=1956)

<QuerySet [#49470 - The Man Who Knew Too Much (1956)]>

In [30]:
# iexact = exact case insensitive
Movie.objects.filter(title__iexact="the man who knew too much")

<QuerySet [#25452 - The Man Who Knew Too Much (1934), #49470 - The Man Who Knew Too Much (1956)]>

In [31]:
# (i)startswith, (i)endswith, (i)contains
Movie.objects.filter(title__istartswith="the man")

<QuerySet [#20142 - The Manxman (1929), #24293 - The Man from Monterey (1933), #25452 - The Man Who Knew Too Much (1934), #25455 - The Man from Utah (1934), #49470 - The Man Who Knew Too Much (1956), #56217 - The Man Who Shot Liberty Valance (1962), #71807 - The Man with the Golden Gun (1974), #73341 - The Man Who Would Be King (1975), #89543 - The Man with One Red Shoe (1985), #243133 - The Man Who Wasn't There (2001)]>

In [32]:
Movie.objects.filter(title__icontains="star")

<QuerySet [#25830 - The Star Packer (1934), #49793 - Star in the Dust (1956), #76759 - Star Wars: Episode IV - A New Hope (1977), #80684 - Star Wars: Episode V - The Empire Strikes Back (1980), #86190 - Star Wars: Episode VI - Return of the Jedi (1983), #95405 - Starry is the Night (1988), #2488496 - Star Wars: Episode VII - The Force Awakens (2015), #2527338 - Star Wars: Episode IX - The Rise of Skywalker (2019)]>

In [35]:
# (i)regex: regular expression
Movie.objects.filter(title__iregex=r"(^| )star( |$)")

<QuerySet [#25830 - The Star Packer (1934), #49793 - Star in the Dust (1956), #76759 - Star Wars: Episode IV - A New Hope (1977), #80684 - Star Wars: Episode V - The Empire Strikes Back (1980), #86190 - Star Wars: Episode VI - Return of the Jedi (1983), #2488496 - Star Wars: Episode VII - The Force Awakens (2015), #2527338 - Star Wars: Episode IX - The Rise of Skywalker (2019)]>

In [36]:
# comparison operator: gt >, gte >=, lt <, lte <=
Movie.objects.filter(year__gt=2018)

<QuerySet [#8079250 - Oppenheimer (2023), #8079251 - Oppenheimer (2023), #8079253 - Oppenheimer (2023), #8079255 - Oppenheimer (2023), #8079256 - Barbie (2023), #1025100 - Gemini Man (2019), #1206885 - Rambo: Last Blood (2019), #1302006 - The Irishman (2019), #1502397 - Bad Boys for Life (2020), #1634106 - Bloodshot (2020), #1979376 - Toy Story 4 (2019), #2527338 - Star Wars: Episode IX - The Rise of Skywalker (2019), #3224458 - A Beautiful Day in the Neighborhood (2019), #7286456 - Joker (2019), #3282076 - Domino (2019), #3513548 - Richard Jewell (2019), #4154796 - Avengers: Endgame (2019), #5113040 - The Secret Life of Pets 2 (2019), #6139732 - Aladdin (2019), #5814534 - Spies in Disguise (2019), '...(remaining elements truncated)...']>

In [44]:
# range: between
movies = list(Movie.objects
              .exclude(year=1982)
              .filter(year__range=(1980,1989))
              .order_by("year", "title")
             )
movies

[#80377 - Any Which Way You Can (1980),
 #80472 - Bronco Billy (1980),
 #80661 - Dressed to Kill (1980),
 #80850 - He Knows You're Alone (1980),
 #82880 - Il pap'occhio (1980),
 #76580 - Qian zuo guai (1980),
 #81398 - Raging Bull (1980),
 #80684 - Star Wars: Episode V - The Empire Strikes Back (1980),
 #80455 - The Blues Brothers (1980),
 #80907 - The Hunter (1980),
 #80031 - Tom Horn (1980),
 #82085 - Blow Out (1981),
 #82398 - For Your Eyes Only (1981),
 #82535 - Hua ji shi dai (1981),
 #82817 - Nighthawks (1981),
 #82869 - Outland (1981),
 #82910 - Piranha II: The Spawning (1981),
 #82971 - Raiders of the Lost Ark (1981),
 #83907 - The Evil Dead (1981),
 #81633 - Time Bandits (1981),
 #83284 - Victory (1981),
 #173893 - Hefty's (1983),
 #86006 - Never Say Never Again (1983),
 #86034 - Octopussy (1983),
 #86250 - Scarface (1983),
 #86190 - Star Wars: Episode VI - Return of the Jedi (1983),
 #86361 - Staying Alive (1983),
 #86383 - Sudden Impact (1983),
 #86491 - Twilight Zone: The M

In [45]:
Movie.objects.filter(duration__isnull=True)

<QuerySet [#8079250 - Oppenheimer (2023), #13444 - Number 13 (1922), #8079256 - Barbie (2023), #77667 - Ha luo, ye gui ren (1978), #173893 - Hefty's (1983), #185446 - Gu lian hua (1985), #4824394 - Terminator Genisys: The YouTube Chronicles (2015), #6344712 - The Square Root (1969)]>

In [48]:
date1 = date(1930,1,1)
date2 = date(1930,12,31)
Person.objects.filter(birthdate__range=(date1,date2))

<QuerySet [#125 - Sean Connery, #142 - Clint Eastwood, #432 - Gene Hackman, #537 - Steve McQueen, #728 - Mario Adorf, #1262 - Ben Gazzara, #1321 - Richard Harris, #1335 - Tippi Hedren, #1687 - Gena Rowlands, #1703 - Maximilian Schell, #1792 - Rod Taylor, #1822 - Robert Wagner, #3679 - Kate Reid, #4877 - Roy Edward Disney, #5162 - Robert Loggia, #5196 - Paul Mazursky, #43484 - Jack Axelrod, #53364 - Chris Barber, #58972 - Dominic Barto, #60789 - Charles Bateman, '...(remaining elements truncated)...']>

In [51]:
reset_queries()
Person.objects.filter(birthdate__year=1930)

<QuerySet [#125 - Sean Connery, #142 - Clint Eastwood, #432 - Gene Hackman, #537 - Steve McQueen, #728 - Mario Adorf, #1262 - Ben Gazzara, #1321 - Richard Harris, #1335 - Tippi Hedren, #1687 - Gena Rowlands, #1703 - Maximilian Schell, #1792 - Rod Taylor, #1822 - Robert Wagner, #3679 - Kate Reid, #4877 - Roy Edward Disney, #5162 - Robert Loggia, #5196 - Paul Mazursky, #43484 - Jack Axelrod, #53364 - Chris Barber, #58972 - Dominic Barto, #60789 - Charles Bateman, '...(remaining elements truncated)...']>

In [52]:
connection.queries

[{'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate" FROM "person" WHERE "person"."birthdate" BETWEEN \'1930-01-01\'::date AND \'1930-12-31\'::date LIMIT 21',
  'time': '0.000'}]

In [61]:
reset_queries()
Person.objects.annotate(year=Extract('birthdate','year')).filter(year=1930)

<QuerySet [#125 - Sean Connery, #142 - Clint Eastwood, #432 - Gene Hackman, #537 - Steve McQueen, #728 - Mario Adorf, #1262 - Ben Gazzara, #1321 - Richard Harris, #1335 - Tippi Hedren, #1687 - Gena Rowlands, #1703 - Maximilian Schell, #1792 - Rod Taylor, #1822 - Robert Wagner, #3679 - Kate Reid, #4877 - Roy Edward Disney, #5162 - Robert Loggia, #5196 - Paul Mazursky, #43484 - Jack Axelrod, #53364 - Chris Barber, #58972 - Dominic Barto, #60789 - Charles Bateman, '...(remaining elements truncated)...']>

In [62]:
connection.queries

[{'sql': 'SELECT "person"."id", "person"."name", "person"."birthdate", EXTRACT(YEAR FROM "person"."birthdate") AS "year" FROM "person" WHERE EXTRACT(YEAR FROM "person"."birthdate") = 1930 LIMIT 21',
  'time': '0.031'}]

### 3.2 - Projections

In [63]:
Movie.objects.values('year')

<QuerySet [{'year': 1915}, {'year': 1916}, {'year': 1917}, {'year': 1917}, {'year': 1917}, {'year': 1917}, {'year': 1918}, {'year': 1918}, {'year': 1925}, {'year': 1928}, {'year': 2023}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1918}, {'year': 1929}, '...(remaining elements truncated)...']>

In [65]:
Movie.objects.filter(title__icontains='star').values('year')

<QuerySet [{'year': 1934}, {'year': 1956}, {'year': 1977}, {'year': 1980}, {'year': 1983}, {'year': 1988}, {'year': 2015}, {'year': 2019}]>

In [67]:
Movie.objects.filter(title__icontains='star').values('title','year')

<QuerySet [{'title': 'The Star Packer', 'year': 1934}, {'title': 'Star in the Dust', 'year': 1956}, {'title': 'Star Wars: Episode IV - A New Hope', 'year': 1977}, {'title': 'Star Wars: Episode V - The Empire Strikes Back', 'year': 1980}, {'title': 'Star Wars: Episode VI - Return of the Jedi', 'year': 1983}, {'title': 'Starry is the Night', 'year': 1988}, {'title': 'Star Wars: Episode VII - The Force Awakens', 'year': 2015}, {'title': 'Star Wars: Episode IX - The Rise of Skywalker', 'year': 2019}]>

In [69]:
# distinct
years = list(Movie.objects.values('year').distinct())
len(years)

107

In [77]:
reset_queries()
q = Person.objects \
        .filter(birthdate__isnull=False) \
        .annotate(year=Extract('birthdate', 'year')) \
        .values('name','year') \
        .order_by('-year','name')
q

<QuerySet [{'name': 'Gary the Dog', 'year': 2012}, {'name': 'Temirlan Blaev', 'year': 2007}, {'name': 'Erin Kellyman', 'year': 1998}, {'name': 'Ross Beadman', 'year': 1998}, {'name': 'Alex Knoll', 'year': 1994}, {'name': 'Chelsea Mather', 'year': 1994}, {'name': 'Keira Wingate', 'year': 1994}, {'name': 'Jett Lucas', 'year': 1993}, {'name': "Helena Collins O'Connor", 'year': 1992}, {'name': 'Chris Adams', 'year': 1990}, {'name': 'Jennifer Lawrence', 'year': 1990}, {'name': 'Keisha Castle-Hughes', 'year': 1990}, {'name': 'Sam Wilkinson', 'year': 1990}, {'name': 'Dalton Abbott', 'year': 1989}, {'name': 'Jake Lloyd', 'year': 1989}, {'name': 'James Henri-Thomas', 'year': 1989}, {'name': 'Samantha Colley', 'year': 1989}, {'name': 'Seth Adkins', 'year': 1989}, {'name': 'Bobby Edner', 'year': 1988}, {'name': 'Chelsea Hamill', 'year': 1988}, '...(remaining elements truncated)...']>

In [78]:
connection.queries

[{'sql': 'SELECT "person"."name", EXTRACT(YEAR FROM "person"."birthdate") AS "year" FROM "person" WHERE "person"."birthdate" IS NOT NULL ORDER BY 2 DESC, "person"."name" ASC LIMIT 21',
  'time': '0.016'}]

In [83]:
# .filter(duration__isnull=True) \
q = Movie.objects \
    .filter(year=2023) \
    .annotate(duration2=Coalesce('duration', 0)) \
    .values('title', 'duration2')
q

<QuerySet [{'title': 'Oppenheimer', 'duration2': 0}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Oppenheimer', 'duration2': 180}, {'title': 'Barbie', 'duration2': 0}]>

In [90]:
Movie.objects \
    .filter(year=1984) \
    .annotate(long_title=Concat('title', 'posterUri')) \
    .values('long_title')

<QuerySet [{'long_title': 'Sword of the Valiant: The Legend of Sir Gawain and the Green Knighthttps://m.media-amazon.com/images/M/MV5BZDc1MDY2YmItYzViMS00YzJiLWJhNDQtNjU5OGI1MTc3NmVmXkEyXkFqcGdeQXVyNjMwMjk0MTQ@._V1_SX101_CR0,0,101,150_.jpg'}, {'long_title': 'Bachelor Partyhttps://m.media-amazon.com/images/M/MV5BMDU1MzM1NmQtMzFkMC00NTFmLWI3MTYtNzE2MzAyMjA2MmIwXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_SX101_CR0,0,101,150_.jpg'}, {'long_title': 'Blood Simplehttps://m.media-amazon.com/images/M/MV5BZmI5YzM1MjItMzFmNy00NGFkLThlMDUtZjZmYTZkM2QxMjU3XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX101_CR0,0,101,150_.jpg'}, {'long_title': 'Body Doublehttps://m.media-amazon.com/images/M/MV5BYzA4MDViNmQtZTk5Yi00ODk2LTliZWEtMTAzYjAzYzQ4YjZlXkEyXkFqcGdeQXVyNDkzNTM2ODg@._V1_SX101_CR0,0,101,150_.jpg'}, {'long_title': "Breakin'https://m.media-amazon.com/images/M/MV5BMDQxNjdlMjctZjRiYi00ODdjLTg4ZTctMTdiNDdlNjk2NjIzXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX101_CR0,0,101,150_.jpg"}, {'long_title': 'City Heathttps://m.media-amazon.com/

In [92]:
Movie.objects \
    .filter(year=1984) \
    .annotate(long_title=Concat('title', V(' | poster = '))) \
    .values('long_title')

<QuerySet [{'long_title': 'Sword of the Valiant: The Legend of Sir Gawain and the Green Knight | poster = '}, {'long_title': 'Bachelor Party | poster = '}, {'long_title': 'Blood Simple | poster = '}, {'long_title': 'Body Double | poster = '}, {'long_title': "Breakin' | poster = "}, {'long_title': 'City Heat | poster = '}, {'long_title': 'Conan the Destroyer | poster = '}, {'long_title': 'Gremlins | poster = '}, {'long_title': 'Indiana Jones and the Temple of Doom | poster = '}, {'long_title': 'Missing in Action | poster = '}, {'long_title': 'Monaco Forever | poster = '}, {'long_title': 'Rhinestone | poster = '}, {'long_title': 'Splash | poster = '}, {'long_title': 'The Terminator | poster = '}, {'long_title': 'Tightrope | poster = '}, {'long_title': 'Heroes Shed No Tears | poster = '}]>